In [1]:
import math
import tensorflow as tf
import numpy as np
import matplotlib
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)
xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/2015eliminated_1.txt',delimiter = ',',dtype = None)

#data_preprocessing
xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

xy_with_noise = xy_with_noise[:,:27]
a_with_noise = xy_with_noise[:,:2]
b_with_noise = xy_with_noise[:,2:]
b_with_noise = MinMaxScaler(b_with_noise)
xy_with_noise = np.hstack((a_with_noise,b_with_noise))


#parameters
seq_length =6
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.001
iterations = 4000

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

#divide data set to train,validation and test set
train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

train_set_with_noise = xy_with_noise[:train_size]
validation_set_with_noise = xy_with_noise[train_size:train_size+validation_size]
test_set_with_noise = xy_with_noise[train_size+validation_size:]

# build data set for rnn
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
validationX_with_noise, validationY_with_noise = build_dataset(validation_set_with_noise,seq_length)
testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])

X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기

with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)

    loss1 =tf.reduce_sum(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)

with tf.variable_scope("rnn2"):
    cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)


#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
x2 = x1+0.3
x3 = x2+0.3
loss_for_graph = np.zeros(iterations)
x4 = np.array(range(0,iterations))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        print("[step: {}] loss: {}".format(i,step_loss1))
        loss_for_graph[i] = step_loss1
        _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        print("[step: {}] loss: {}".format(i,step_loss2))
        
    test_predict = sess.run(Y_pred, feed_dict = {X1:validationX})
    test_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:validationX_with_noise})

    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: test_predict})
    print("RMSE: {}".format(rmse_val))
   # print("pred: {}".format(test_predict[-1,:]))
    #print("real: {}".format(testY[-1,:]))
    #print("noise: {}".format(eliminate_noise_pred[-1,:]))
    
#    plt.bar(x1,test_predict[-1,:],label = 'predict',color ='b',width = 0.1)
  #  plt.bar(x2,testY[-1,:],label = 'real',color ='g',width = 0.1)
    #plt.bar(x3,eliminate_noise_pred[-1,:],label = 'noise',color ='g',width = 0.1)
    plt.plot(x4,loss_for_graph)
    #plt.legend()
    plt.show()



Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 19893.150390625
[step: 0] loss: 0.3186686038970947
[step: 1] loss: 19067.734375
[step: 1] loss: 0.30558547377586365
[step: 2] loss: 18279.875
[step: 2] loss: 0.2931113541126251
[step: 3] loss: 17528.69140625
[step: 3] loss: 0.2812255918979645
[step: 4] loss: 16812.955078125
[step: 4] loss: 0.26991498470306396
[step: 5] loss: 16131.158203125
[step: 5] loss: 0.25920605659484863
[step: 6] loss: 15481.6103515625
[step: 6] loss: 0.2492152601480484
[step: 7] loss: 14862.478515625
[step: 7] loss: 0.23991283774375916
[step: 8] loss: 14271.810546875
[step: 8] loss: 0.23092059791088104
[step: 9] loss: 13707.6875
[step: 9] loss: 0.22228389978408813
[step: 10] loss: 13168.359375
[step: 10] loss: 0.2140980362892151
[step: 11] loss: 12652.1640625
[step: 11

[step: 111] loss: 835.6884765625
[step: 111] loss: 0.015914304181933403
[step: 112] loss: 826.9771728515625
[step: 112] loss: 0.01583445630967617
[step: 113] loss: 818.4353637695312
[step: 113] loss: 0.01575656607747078
[step: 114] loss: 810.056396484375
[step: 114] loss: 0.015680545940995216
[step: 115] loss: 801.8283081054688
[step: 115] loss: 0.015606320463120937
[step: 116] loss: 793.755859375
[step: 116] loss: 0.015533847734332085
[step: 117] loss: 785.8515014648438
[step: 117] loss: 0.015463097020983696
[step: 118] loss: 778.1062622070312
[step: 118] loss: 0.015393983572721481
[step: 119] loss: 770.498779296875
[step: 119] loss: 0.015326494351029396
[step: 120] loss: 763.010986328125
[step: 120] loss: 0.015260529704391956
[step: 121] loss: 755.6219482421875
[step: 121] loss: 0.015196087770164013
[step: 122] loss: 748.3184204101562
[step: 122] loss: 0.015133103355765343
[step: 123] loss: 741.1001586914062
[step: 123] loss: 0.015071611851453781
[step: 124] loss: 733.966552734375
[s

[step: 223] loss: 0.011162039823830128
[step: 224] loss: 323.44622802734375
[step: 224] loss: 0.011127067729830742
[step: 225] loss: 321.53778076171875
[step: 225] loss: 0.011094952002167702
[step: 226] loss: 319.6458435058594
[step: 226] loss: 0.011063022539019585
[step: 227] loss: 317.772705078125
[step: 227] loss: 0.011028674431145191
[step: 228] loss: 315.9273681640625
[step: 228] loss: 0.010999062098562717
[step: 229] loss: 314.112548828125
[step: 229] loss: 0.01096464041620493
[step: 230] loss: 312.3265686035156
[step: 230] loss: 0.010935207828879356
[step: 231] loss: 310.566162109375
[step: 231] loss: 0.010902639478445053
[step: 232] loss: 308.82781982421875
[step: 232] loss: 0.010871323756873608
[step: 233] loss: 307.10894775390625
[step: 233] loss: 0.010841521434485912
[step: 234] loss: 305.4039001464844
[step: 234] loss: 0.010809320025146008
[step: 235] loss: 303.7149658203125
[step: 235] loss: 0.010780150070786476
[step: 236] loss: 302.0397644042969
[step: 236] loss: 0.01074

[step: 334] loss: 208.27456665039062
[step: 334] loss: 0.008796237409114838
[step: 335] loss: 207.7695770263672
[step: 335] loss: 0.00878418143838644
[step: 336] loss: 207.27243041992188
[step: 336] loss: 0.008772255852818489
[step: 337] loss: 206.7857208251953
[step: 337] loss: 0.00876044575124979
[step: 338] loss: 206.31143188476562
[step: 338] loss: 0.008748720400035381
[step: 339] loss: 205.85809326171875
[step: 339] loss: 0.008737083524465561
[step: 340] loss: 205.42576599121094
[step: 340] loss: 0.008725551888346672
[step: 341] loss: 205.03073120117188
[step: 341] loss: 0.008714125491678715
[step: 342] loss: 204.63812255859375
[step: 342] loss: 0.008702811785042286
[step: 343] loss: 204.24407958984375
[step: 343] loss: 0.008691626600921154
[step: 344] loss: 203.7297821044922
[step: 344] loss: 0.008680528961122036
[step: 345] loss: 203.1422119140625
[step: 345] loss: 0.008669549599289894
[step: 346] loss: 202.53485107421875
[step: 346] loss: 0.008658667095005512
[step: 347] loss: 

[step: 444] loss: 0.007942479103803635
[step: 445] loss: 172.52330017089844
[step: 445] loss: 0.007937511429190636
[step: 446] loss: 172.30874633789062
[step: 446] loss: 0.007932553067803383
[step: 447] loss: 172.09658813476562
[step: 447] loss: 0.007927648723125458
[step: 448] loss: 171.8876953125
[step: 448] loss: 0.007922790944576263
[step: 449] loss: 171.6846466064453
[step: 449] loss: 0.007917962968349457
[step: 450] loss: 171.49270629882812
[step: 450] loss: 0.007913175038993359
[step: 451] loss: 171.32167053222656
[step: 451] loss: 0.007908430881798267
[step: 452] loss: 171.198486328125
[step: 452] loss: 0.007903714664280415
[step: 453] loss: 171.1573486328125
[step: 453] loss: 0.007899043150246143
[step: 454] loss: 171.3051300048828
[step: 454] loss: 0.007894409820437431
[step: 455] loss: 171.59942626953125
[step: 455] loss: 0.007889819331467152
[step: 456] loss: 172.07046508789062
[step: 456] loss: 0.007885261438786983
[step: 457] loss: 171.67373657226562
[step: 457] loss: 0.0

...

[step: 3473] loss: 0.006392274983227253
[step: 3474] loss: 45.166175842285156
[step: 3474] loss: 0.006392131093889475
[step: 3475] loss: 45.848209381103516
[step: 3475] loss: 0.006391997914761305
[step: 3476] loss: 44.82134246826172
[step: 3476] loss: 0.006391859147697687
[step: 3477] loss: 44.460052490234375
[step: 3477] loss: 0.006391719449311495
[step: 3478] loss: 45.23445510864258
[step: 3478] loss: 0.006391582544893026
[step: 3479] loss: 45.075599670410156
[step: 3479] loss: 0.00639144703745842
[step: 3480] loss: 44.371707916259766
[step: 3480] loss: 0.006391302216798067
[step: 3481] loss: 44.597900390625
[step: 3481] loss: 0.006391169503331184
[step: 3482] loss: 44.9675178527832
[step: 3482] loss: 0.0063910274766385555
[step: 3483] loss: 44.55812454223633
[step: 3483] loss: 0.0063908942975103855
[step: 3484] loss: 44.28059768676758
[step: 3484] loss: 0.006390745285898447
[step: 3485] loss: 44.6151123046875
[step: 3485] loss: 0.006390609312802553
[step: 3486] loss: 44.658084869384

[step: 3579] loss: 0.006378684658557177
[step: 3580] loss: 43.188533782958984
[step: 3580] loss: 0.006378146819770336
[step: 3581] loss: 43.17815017700195
[step: 3581] loss: 0.006377996876835823
[step: 3582] loss: 43.16778564453125
[step: 3582] loss: 0.006378143560141325
[step: 3583] loss: 43.15739440917969
[step: 3583] loss: 0.006378231104463339
[step: 3584] loss: 43.14692687988281
[step: 3584] loss: 0.006378039717674255
[step: 3585] loss: 43.13652038574219
[step: 3585] loss: 0.006377647165209055
[step: 3586] loss: 43.12611389160156
[step: 3586] loss: 0.006377323996275663
[step: 3587] loss: 43.11576843261719
[step: 3587] loss: 0.0063772001303732395
[step: 3588] loss: 43.10540771484375
[step: 3588] loss: 0.006377220153808594
[step: 3589] loss: 43.095062255859375
[step: 3589] loss: 0.006377204321324825
[step: 3590] loss: 43.08476638793945
[step: 3590] loss: 0.006377041805535555
[step: 3591] loss: 43.07440948486328
[step: 3591] loss: 0.006376778241246939
[step: 3592] loss: 43.06403350830

[step: 3686] loss: 0.006364389788359404
[step: 3687] loss: 42.67330551147461
[step: 3687] loss: 0.006364266853779554
[step: 3688] loss: 42.34680938720703
[step: 3688] loss: 0.006364138331264257
[step: 3689] loss: 42.308799743652344
[step: 3689] loss: 0.006364012602716684
[step: 3690] loss: 42.52480697631836
[step: 3690] loss: 0.006363890133798122
[step: 3691] loss: 42.4593505859375
[step: 3691] loss: 0.0063637662678956985
[step: 3692] loss: 42.24198913574219
[step: 3692] loss: 0.006363654043525457
[step: 3693] loss: 42.28820037841797
[step: 3693] loss: 0.006363534834235907
[step: 3694] loss: 42.41081237792969
[step: 3694] loss: 0.006363433785736561
[step: 3695] loss: 42.30793762207031
[step: 3695] loss: 0.006363346707075834
[step: 3696] loss: 42.18241882324219
[step: 3696] loss: 0.006363286636769772
[step: 3697] loss: 42.24159240722656
[step: 3697] loss: 0.00636327313259244
[step: 3698] loss: 42.299564361572266
[step: 3698] loss: 0.00636333180591464
[step: 3699] loss: 42.21160125732422

[step: 3793] loss: 41.25989532470703
[step: 3793] loss: 0.006351449992507696
[step: 3794] loss: 41.25102615356445
[step: 3794] loss: 0.00635133171454072
[step: 3795] loss: 41.2420654296875
[step: 3795] loss: 0.0063512008637189865
[step: 3796] loss: 41.233150482177734
[step: 3796] loss: 0.0063510797917842865
[step: 3797] loss: 41.22422409057617
[step: 3797] loss: 0.006350950337946415
[step: 3798] loss: 41.215293884277344
[step: 3798] loss: 0.006350819021463394
[step: 3799] loss: 41.20645523071289
[step: 3799] loss: 0.006350707728415728
[step: 3800] loss: 41.19754409790039
[step: 3800] loss: 0.006350589916110039
[step: 3801] loss: 41.188594818115234
[step: 3801] loss: 0.006350474897772074
[step: 3802] loss: 41.17972183227539
[step: 3802] loss: 0.006350349169224501
[step: 3803] loss: 41.17086410522461
[step: 3803] loss: 0.0063502308912575245
[step: 3804] loss: 41.161949157714844
[step: 3804] loss: 0.006350105628371239
[step: 3805] loss: 41.153079986572266
[step: 3805] loss: 0.006349984090

[step: 3901] loss: 40.472801208496094
[step: 3901] loss: 0.006339132785797119
[step: 3902] loss: 40.598472595214844
[step: 3902] loss: 0.006338674575090408
[step: 3903] loss: 40.63993835449219
[step: 3903] loss: 0.006338873878121376
[step: 3904] loss: 40.49414825439453
[step: 3904] loss: 0.006339315790683031
[step: 3905] loss: 40.43962478637695
[step: 3905] loss: 0.006339491810649633
[step: 3906] loss: 40.52854537963867
[step: 3906] loss: 0.006339231040328741
[step: 3907] loss: 40.532981872558594
[step: 3907] loss: 0.006338647101074457
[step: 3908] loss: 40.42796325683594
[step: 3908] loss: 0.0063381134532392025
[step: 3909] loss: 40.40226745605469
[step: 3909] loss: 0.006337872240692377
[step: 3910] loss: 40.461124420166016
[step: 3910] loss: 0.006337922066450119
[step: 3911] loss: 40.451866149902344
[step: 3911] loss: 0.0063380650244653225
[step: 3912] loss: 40.37855529785156
[step: 3912] loss: 0.006338111590594053
[step: 3913] loss: 40.3623161315918
[step: 3913] loss: 0.006337942555